# Robotics, Vision & Control 3e: for Python
## Chapter 14: Using Multiple Images

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install matplotlib
    !pip install machinevision-toolbox-python
    !pip install --no-deps rvc3python
    COLAB = True
except:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
from IPython.core.display import HTML

import RVC3 as rvc
import sys, os.path
sys.path.append(os.path.join(rvc.__path__[0], 'examples'))

import numpy as np
from scipy import linalg, stats
import matplotlib.pyplot as plt
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import *



# 14.1 Point Feature Correspondence


In [ ]:
view1 = Image.Read("eiffel-1.png", mono=True)
view2 = Image.Read("eiffel-2.png", mono=True)

In [ ]:
hf = view1.Harris(nfeat=150)
view1.disp(block=None, darken=True); hf.plot();

In [ ]:
sf = view1.SIFT().sort().filter(minscale=10)[:150]
view1.disp(block=None, darken=True); sf.plot(filled=True, color="y", alpha=0.3)

In [ ]:
hf[0].descriptor.shape

In [ ]:
hf[0].distance(hf[1], metric="ncc")

In [ ]:
sf[0].descriptor.shape

In [ ]:
sf[0].distance(sf[1], metric="L2")

In [ ]:
sf1 = view1.SIFT()
sf2 = view2.SIFT()

In [ ]:
matches = sf1.match(sf2);
len(matches)

In [ ]:
matches[:5].list()

In [ ]:
matches.subset(100).plot(color="yellow", block=None)

In [ ]:
c = matches.correspondence();
c[:, :5]

In [ ]:
plt.hist(matches.distance, cumulative=True, density=True);

In [ ]:
m = sf1.match(sf2, thresh=20);

In [ ]:
m = sf1.match(sf2, sort=True)[:10];

In [ ]:
m = sf1.match(sf2, ratio=0.8)

In [ ]:
m = sf1.match(sf2, crosscheck=True)

# 14.2 Geometry of Multiple Views


In [ ]:
camera1 = CentralCamera(name="camera 1", f=0.002, imagesize=1000, 
                        rho=10e-6, pose=SE3.Tx(-0.1)*SE3.Ry(0.4))

In [ ]:
camera2 = CentralCamera(name="camera 2", f=0.002, imagesize=1000, 
                        rho=10e-6, pose=SE3.Tx(0.1)*SE3.Ry(-0.4))

In [ ]:
ax = plotvol3([-0.4, 0.6, -0.5, 0.5, -0.2, 1]);
camera1.plot(ax=ax, scale=0.15, shape="camera", frame=True, color="blue");
camera2.plot(ax=ax, scale=0.15, shape="camera", frame=True, color="red");
P=[0.5, 0.1, 0.8];
plot_sphere(0.03, P, color="blue");

In [ ]:
p1 = camera1.plot_point(P)
p2 = camera2.plot_point(P)

In [ ]:
e1 = camera1.plot_point(camera2.centre, "kd")
e2 = camera2.plot_point(camera1.centre, "kd")

## 14.2.1 The Fundamental Matrix


In [ ]:
F = camera1.F(camera2)

In [ ]:
e2h(p2).T @ F @ e2h(p1)

In [ ]:
np.linalg.matrix_rank(F)

In [ ]:
e1h = linalg.null_space(F);
e1h.T

In [ ]:
e1 = h2e(e1h)

In [ ]:
e2h = linalg.null_space(F.T);
e2 = h2e(e2h)

In [ ]:
plt.clf()
with plt.ioff():
    camera2.plot_epiline(F, p1, color="red");
    camera2.plot_point(camera1.centre, "kd");  # show the epipole
    camera2.plot_point(P);                     # show the point

In [ ]:
plt.clf()
with plt.ioff():
    camera1.plot_epiline(F.T, p2, color="red");
    camera1.plot_point(camera2.centre, "kd");  # show the epipole
    camera1.plot_point(P);                     # show the point


## 14.2.2 The Essential Matrix


In [ ]:
E = camera1.E(F)

In [ ]:
T_1_2 = camera1.decomposeE(E);
T_1_2.printline(orient="camera")

In [ ]:
T_1_2_true = camera1.pose.inv() * camera2.pose;
T_1_2_true.printline(orient="camera")

In [ ]:
T_1_2_true.t / np.linalg.norm(T_1_2_true.t)

In [ ]:
Q = [0, 0, 10];

In [ ]:
camera1.project_point(Q).T

In [ ]:
for T in T_1_2:
 print(camera1.project_point(Q, pose=T).T)

In [ ]:
T = camera1.decomposeE(E, Q);
T.printline(orient="camera")

## 14.2.3 Estimating the Fundamental Matrix from Real Image Data


In [ ]:
P = np.random.uniform(low=-1, high=1, size=(3, 10)) + np.c_[0, 0, 3].T;

In [ ]:
p1 = camera1.project_point(P);
p2 = camera2.project_point(P);

In [ ]:
F, resid = CentralCamera.points2F(p1, p2)
resid

In [ ]:
np.linalg.matrix_rank(F)

In [ ]:
plt.clf()
with plt.ioff():
    camera2.plot_point(P);
    print(plt.isinteractive())
    camera2.plot_epiline(F, p1, color="red")
    camera2.plot_point(camera1.centre, "kd")

In [ ]:
p2[:,[5, 6]] = p2[:,[6, 5]];

In [ ]:
_, resid = CentralCamera.points2F(p1, p2);
resid

In [ ]:
CentralCamera.epidist(F, p1[:, 0], p2[:,0])
CentralCamera.epidist(F, p1[:, 5], p2[:,5])

In [ ]:
F, resid, inliers = CentralCamera.points2F(p1, p2, method="ransac", 
                                           confidence=0.99, seed=0);
resid

In [ ]:
inliers

In [ ]:
F, resid, inliers = CentralCamera.points2F(matches.p1, matches.p2, 
                                           method="ransac", confidence=0.99);
resid
sum(inliers) / len(inliers)

In [ ]:

sum(~inliers)

In [ ]:
F, resid = matches.estimate(CentralCamera.points2F, method="ransac", 
                            confidence=0.99, seed=0);

In [ ]:
matches
matches[:10].list()

In [ ]:
matches.inliers.subset(100).plot(color="g", block=None);

In [ ]:
matches.outliers.subset(100).plot(color="red")

In [ ]:
camera = CentralCamera();
with plt.ioff():
    camera.disp(view1, block=None);
    camera.plot_epiline(F.T, matches.inliers.subset(20).p2, color="black");
    epipole = h2e(linalg.null_space(F))
    camera.plot_point(epipole, "wd");
plt.show()

#### Excurse 14.3

In [ ]:
if COLAB:
    !pip install --no-deps rvc3python

from RVC3.examples import ransac_line

x = np.arange(11);
y = 3 * x - 10;
nbad = 4;
np.random.seed(1)  # set the random number generator seed
bad = np.random.choice(len(x), nbad, replace=False)
y[bad] = y[bad] + np.random.rand(nbad) * 10
plt.plot(x, y, 'o')

m, c, *_ = stats.linregress(x, y)
plt.plot(x, m * x + c, 'r--');
params, inliers = ransac_line.ransac_line(x, y)
params
inliers


## 14.2.4 Planar Homography


In [ ]:
T_grid = SE3.Tz(1) * SE3.Rx(0.1) * SE3.Ry(0.2);
P = mkgrid(3, 1.0, pose=T_grid);

In [ ]:
p1 = camera1.plot_point(P, "o");
p2 = camera2.plot_point(P, "o");

In [ ]:
H, resid = CentralCamera.points2H(p1, p2)
H

In [ ]:
p2b = homtrans(H, p1);

In [ ]:
with plt.ioff():
    camera2.plot_point(p2b, "+");
    camera2.plot_point(P, "o");  # show original points

In [ ]:
p1b = homtrans(np.linalg.inv(H), p1);

In [ ]:
Q = np.array([
  [-0.2302,   -0.0545,    0.2537],
  [ 0.3287,    0.4523,    0.6024],
  [ 0.4000,    0.5000,    0.6000] ]);

In [ ]:
plotvol3([-1, 1, -1, 1, 0, 2]);
plot_sphere(0.05, P, color="blue");
plot_sphere(0.05, Q, color="red");
camera1.plot(color="blue", frame=True);
camera2.plot(color="red", frame=True);

In [ ]:
p1 = camera1.plot_point(np.hstack((P, Q)), "o");

In [ ]:
p2 = camera2.plot_point(np.hstack((P, Q)), "o");

In [ ]:
p2h = homtrans(H, p1);

In [ ]:
with plt.ioff():
    camera2.plot_point(p2h, "+");
    camera2.plot_point(p2);    # show projected points
plt.show()

In [ ]:
np.linalg.norm(homtrans(H, p1) - p2, axis=0)

In [ ]:
H, resid, inliers = CentralCamera.points2H(p1, p2, method="ransac");
resid
inliers

In [ ]:
T, normals = camera1.decomposeH(H);
T.printline(orient="camera")

In [ ]:
(camera1.pose.inv() * camera2.pose).printline(orient="camera")

In [ ]:
camera1.pose.inv() * T_grid

In [ ]:
normals[1].T

In [ ]:
walls_l = Image.Read("walls-l.png", reduce=2);
walls_r = Image.Read("walls-r.png", reduce=2);

In [ ]:
sf_l = walls_l.SIFT();
sf_r = walls_r.SIFT();

In [ ]:
matches = sf_l.match(sf_r);

In [ ]:
H, resid = matches.estimate(CentralCamera.points2H, confidence=0.9, seed=0)
matches

In [ ]:
walls_l.disp(block=None);
plot_point(matches.inliers.p1, "r.");

In [ ]:
not_plane = matches.outliers;

# 14.3 Sparse Stereo


## 14.3.1 3D Triangulation


In [ ]:
matches = sf_l.match(sf_r)
F, resid = matches.estimate(CentralCamera.points2F, confidence=0.99, seed=0);

In [ ]:
matches = matches.inliers  # keep only the inliers

In [ ]:
camera = CentralCamera();
camera.disp(walls_l, block=None);
camera.plot_epiline(F.T, matches.subset(40).p2, "yellow");

In [ ]:
f = walls_l.metadata("FocalLength")

In [ ]:
name = walls_l.metadata("Model")

In [ ]:
camera = CentralCamera(name=name, imagesize=walls_l.shape, 
                       f=f/1000, rho=2*1.5e-6)

In [ ]:
E = camera.E(F)

In [ ]:
T_1_2 = camera.decomposeE(E, [0, 0, 10]);
T_1_2.printline(orient="camera")

In [ ]:
t = T_1_2.t;
s = 0.3 / t[0]  # estimate of translation scale factor
T_1_2.t = s * t  # scaled translation
T_1_2.printline(orient="camera")

In [ ]:
ray1 = camera.ray(matches[0].p1)

In [ ]:
ray2 = camera.ray(matches[0].p2, pose=T_1_2)

In [ ]:
P, e = ray1.closest_to_line(ray2);
P

In [ ]:
e

In [ ]:
ray1 = camera.ray(matches.p1);
ray2 = camera.ray(matches.p2, pose=T_1_2);

In [ ]:
len(ray1)

In [ ]:
P, e = ray1.closest_to_line(ray2);
P.shape

In [ ]:
z = P[2, :];
z.mean()

In [ ]:
np.median(e)
e.max()

In [ ]:
plotvol3();
plt.plot(P[0,:], P[1,:], P[2,:], '.', markersize=2);

In [ ]:
if not COLAB:
    walls_pcd = PointCloud(P)
    walls_pcd.transform(SE3.Rx(pi));  # make y-axis upward
    walls_pcd.disp(block=True)
    walls_pcd = walls_pcd.remove_outlier(nb_points=10, radius=0.2)

In [ ]:
if not COLAB:
  colors = []
  for m in matches:
    colors.append(walls_l.image[int(m.p1[1]), int(m.p1[0]), :])
  pcd = SE3.Rx(pi) * PointCloud(P, colors=np.array(colors).T)
  pcd.disp(block=True)

In [ ]:
p1_reproj = camera.project_point(P[:, 0]);
p2_reproj = camera.project_point(P[:, 0], pose=T_1_2);

In [ ]:
(p1_reproj - matches[0].p1).T
(p2_reproj - matches[0].p2).T

In [ ]:
bundle = BundleAdjust(camera);

In [ ]:
view0 = bundle.add_view(SE3(), fixed=True);
view1 = bundle.add_view(SE3.Tx(0.3));

In [ ]:
for (Pj, mj) in zip(P[:, ::4].T, matches[::4]):
  landmark = bundle.add_landmark(Pj)             # add vertex
  bundle.add_projection(view0, landmark, mj.p1)  # add edge
  bundle.add_projection(view1, landmark, mj.p2)  # add edge

In [ ]:
bundle

In [ ]:
bundle.plot()

In [ ]:
x = bundle.getstate();
x.shape

In [ ]:
x[6:12]

In [ ]:
x[12:15]

In [ ]:
bundle.errors(x)

p, A, B = camera.derivatives(t, r, P);

In [ ]:
x_new, resid = bundle.optimize(x);

In [ ]:
bundle.setstate(x_new);

In [ ]:
bundle.views[1].pose.printline(orient="camera")

In [ ]:
T_1_2.printline(orient="camera")

In [ ]:
bundle.landmarks[0].P

In [ ]:
e = np.sqrt(bundle.getresidual());
e.shape

In [ ]:
np.median(e, axis=1)

In [ ]:
np.max(e, axis=1)

# 14.4 Dense Stereo Matching


In [ ]:
rocks_l = Image.Read("rocks2-l.png", reduce=2)
rocks_r = Image.Read("rocks2-r.png", reduce=2)

rocks_l.stdisp(rocks_r)

In [ ]:
disparity, *_ = rocks_l.stereo_simple(rocks_r, hw=3, drange=[40, 90]);

In [ ]:
disparity.disp(colorbar=True);

In [ ]:
disparity, similarity, DSI = rocks_l.stereo_simple(rocks_r, hw=3, drange=[40, 90])

In [ ]:
DSI.shape

In [ ]:
np.argmax(DSI, axis=2);

In [ ]:
similarity_values = np.max(DSI, axis=2);

In [ ]:
plt.plot(DSI[439, 138, :], "o-");

## 14.4.1 Peak Refinement


In [ ]:
disparity_refined, A = Image.DSI_refine(DSI)

## 14.4.2 Stereo Failure Modes


### 14.4.2.1 Multiple peaks


### 14.4.2.2 Weak matching


In [ ]:
similarity.disp();

In [ ]:
similarity.choose("blue", similarity < 0.6).disp();

In [ ]:
plt.hist(similarity.view1d(), 100, (0, 1), cumulative=True, density=True);

### 14.4.2.3 Broad peak


### 14.4.2.4 Quantifying Failure Modes


In [ ]:
status = np.ones(disparity.shape);

In [ ]:
U, V = disparity.meshgrid()
status[np.isnan(disparity.image)] = 5   # no similarity computed
status[U <= 90] = 2                     # no overlap
status[similarity.image < 0.6] = 3      # weak match
status[A.image >= -0.1] = 4             # broad peak

In [ ]:
plt.imshow(status);

In [ ]:
(status == 1).sum() / status.size * 100

In [ ]:
disparity_valid = disparity.choose(0, status!=1)

### 14.4.2.5 Slicing the DSI


In [ ]:
Image(DSI[100, :, :].T).disp();

### 14.4.2.6 Summary


### 14.4.2.7 Advanced Stereo Matching


In [ ]:
disparity_BM = rocks_l.stereo_BM(rocks_r, hw=3, drange=[40, 90], speckle=(200, 2))
disparity_BM.disp();

In [ ]:
rocks_l.stereo_SGBM(rocks_r, hw=3, drange=[40, 90], speckle=(200, 2)).disp();

### 14.4.2.8 3D Reconstruction


In [ ]:
di = disparity_BM.image * 2 + 274;

In [ ]:
U, V = disparity_BM.meshgrid();
u0, v0 = disparity.centre;
f = 3740;   # pixels, according to Middlebury website
b = 0.160;  # m, according to Middlebury website
X = b * (U - u0) / di; Y = b * (V - v0) / di; Z = f * b / di;

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(X, Y, Z)
ax.view_init(-100, -100)

In [ ]:
if not COLAB:
    cam = CentralCamera(f=f, imagesize=rocks_l.shape);
    pcd = PointCloud(Z, image=rocks_l, camera=cam, depth_trunc=1.9)
    pcd *= SE3.Rx(pi);  # make y-axis upward
    pcd.disp(block=True)

## 14.4.3 Image Rectification


In [ ]:
walls_l = Image.Read('walls-l.png', reduce=2)
walls_r = Image.Read('walls-r.png', reduce=2)

In [ ]:
sf_l = walls_l.SIFT()
sf_r = walls_r.SIFT()

In [ ]:
matches = sf_l.match(sf_r);

In [ ]:
F, resid = matches.estimate(CentralCamera.points2F, 
                            method="ransac", confidence=0.95);

In [ ]:
H_l, H_r = walls_l.rectify_homographies(matches, F)

In [ ]:
walls_l_rect = walls_l.warp_perspective(H_l)
walls_r_rect = walls_r.warp_perspective(H_r)

In [ ]:
if COLAB:
    walls_l_rect.disp()
    walls_r_rect.disp()
else:
    walls_l_rect.stdisp(walls_r_rect)

In [ ]:
walls_l_rect.stereo_SGBM(walls_r_rect, hw=7, drange=[180, 530], speckle=(50, 2)).disp();

# 14.5 Anaglyphs


In [ ]:
walls_l.anaglyph(walls_r, "rc").disp();

# 14.7 Point Clouds


In [ ]:
if not COLAB:
    bunny_pcd = PointCloud.Read('data/bunny.ply')
    bunny_pcd.disp(block=True)
    pcd = bunny_pcd.voxel_grid(voxel_size=0.01).disp(block=True)
    pcd = bunny_pcd.downsample_voxel(voxel_size=0.01)
    pcd.normals(radius=0.1, max_nn=30)
    pcd.disp(block=True)

## 14.7.1 Fitting a Plane


In [ ]:
if not COLAB:
    pcd = walls_pcd
    # plane, plane_pcd, pcd = pcd.segment_plane(distance_threshold=0.05, seed=0)
    plane, plane_pcd, pcd = pcd.segment_plane(distance_threshold=0.05)
    print(plane)

    print(plane_pcd)

    # plane, plane_pcd, pcd = pcd.segment_plane(distance_threshold=0.05, seed=0)
    plane, plane_pcd, pcd = pcd.segment_plane(distance_threshold=0.05)
    print(plane)

## 14.7.2 Matching Two Sets of Points


In [ ]:
if not COLAB:
    model = bunny_pcd.downsample_random(0.1, seed=0)

    data = SE3.Trans(0.3, 0.4, 0.5) * SE3.Rz(50, unit="deg") * bunny_pcd.downsample_random(0.05, seed=-1);

    model.paint([0, 0, 1])  # blue
    data.paint([1, 0, 0])   # red
    (model + data).disp(block=True)

    T, status = model.ICP(data, max_correspondence_distance=1, 
                    max_iteration=2000, relative_fitness=0, relative_rmse=0)
    T.printline()

    (model + T.inv() * data).disp(block=True)

# 14.8 Applications


## 14.8.1 Perspective Correction


In [ ]:
notredame = Image.Read("notre-dame.png");
notredame.disp();

In [ ]:
# this won't work with Jupyter, use the pre-picked points defined in the next cell
# picked_points = plt.ginput(4);
# p1 = np.array(picked_points).T;

In [ ]:
# coordinates of 4 points that mark the corners of a rectangle on the face of
# the building
p1 = np.array([
        [ 44.1364,   94.0065,  537.8506,  611.8247], 
        [377.0654,  152.7850,  163.4019,  366.4486]]);

In [ ]:
notredame.disp(block=None);
plot_polygon(p1, filled=True, color="y", alpha=0.4, linewidth=2);
plot_point(p1, "yo");
mn = p1.min(axis=1);
mx = p1.max(axis=1);
p2 = np.array([[mn[0], mn[0], mx[0], mx[0]], [mx[1], mn[1], mn[1], mx[1]]]);
plot_polygon(p2, "k--", close=True, linewidth=2);

In [ ]:
H, _ = CentralCamera.points2H(p1, p2, method="leastsquares")
H

In [ ]:
notredame.warp_perspective(H).disp();

In [ ]:
f = notredame.metadata("FocalLength")

In [ ]:
cam = CentralCamera(imagesize=notredame.shape, f=f/1000, sensorsize=[7.18e-3, 5.32e-3])

In [ ]:
pose, normals = cam.decomposeH(H)
pose.printline(orient="camera")

In [ ]:
normals[0].T

## 14.8.2 Image Mosaicing


In [ ]:
images = ImageCollection("mosaic/aerial2-*.png", mono=True);

In [ ]:
composite = Image.Zeros(2_000, 2_000)

In [ ]:
composite.paste(images[0], (0, 0));
composite.disp();

In [ ]:
next_image = images[1]
sf_c = composite.SIFT()
sf_next= next_image.SIFT()
match = sf_c.match(sf_next);

In [ ]:
H, _ = match.estimate(CentralCamera.points2H, "ransac", confidence=0.99);
H

In [ ]:
tile, topleft, corners = next_image.warp_perspective(H, inverse=True, tile=True)

In [ ]:
composite.paste(tile, topleft, method="blend");
composite.disp();

## 14.8.3 Visual Odometry


In [ ]:
import mvtbdata.mvtb_load_image_data

left = ZipArchive("bridge-l.zip", filter="*.pgm", mono=True, dtype="uint8", 
                  maxintval=4095, roi=[20, 750, 20, 480]);
len(left)

In [ ]:
import mvtbdata.mvtb_load_image_data

In [ ]:
if not COLAB:  # simple animations do not work properly in Jupyter
  for image in left:
    #image.disp(reuse=True, block=None)
    image.disp(reuse=True, fps=10, title="bridge-l", matplotlib=False)
  cv_destroy_window("bridge-l")

<div class="note">

The commented out code produces an animation for a Python script, however, using Jupyter it produces a set of separate images.  Use OpenCV instead (`matplotlib=False`) to display the animation in a separate window.
</div>

In [ ]:
for image in left:
   features = image.ORB(nfeatures=20)         # compute ORB features
   image = image.colorize()                   # create RGB image
   features.draw2(image, color='y');          # render ORB features into image
   # image.disp(reuse=True, fps=20, matplotlib=False) # display the image
   # features.plot()                          # overlay the features
   # pause(0.05)

   features.draw2(image, color='y');          # render ORB features into image
   image.disp(reuse=True, fps=20, matplotlib=False) # display the image

In [ ]:
right = ZipArchive("bridge-r.zip", mono=True, dtype="uint8", 
                   maxintval=4095, roi=[20, 750, 20, 480]);

In [ ]:
%run -m visodom

In [ ]:
ts = np.loadtxt(left.open("timestamps.dat"));

In [ ]:
plt.plot(np.diff(ts));